# app.php

```php
<?php

$app->withFacades();

$app->withEloquent();

$app->register(Flipbox\LumenGenerator\LumenGeneratorServiceProvider::Class);

$app->routeMiddleware([
    'auth' => App\Http\Middleware\Authenticate::class,
]);

$app->register(App\Providers\AuthServiceProvider::class);

?>
```



# UserController.php

```php
<?php

use Illuminate\Support\Facades\DB;

class UserController extends Controller
{
    public function index()
    {
        $query = DB::connection('mysql')->table('users')->get();
        return response()->json($query, 200);
    }
}

?>
```

# Authenticate.php

```php
<?php

use Illuminate\Support\Facades\DB;

public function handle($request, Closure $next, $guard = null)
    {
        if ($this->auth->guard($guard)->guest()) {
            $token = $request->header('rivai');
            if ($token) {
                $token = trim(str_replace('Bearer', '', $token));
                //$user = User::where('password', $token)->first();
                $user = DB::connection('mysql')->table('users')->where('password', $token)->first();
                dd($user);
                if (!$user) {
                    return response()->json([
                        'success' => false,
                        'message' => 'Permission Not Allowed'
                    ], 403);
                }
                $this->auth->setUser($user);
            } else {
                return response()->json([
                    'success' => false,
                    'message' => 'Not Authorized'
                ], 401);
            }
        }

        return $next($request);
    }

?>
```

# User.php

```php
<?php

class User extends Model
{
    protected $connection = 'mysql';
    protected $fillable = [
        'username'
    ];

    protected $hidden = [
        'password'
    ];
}

?>
```

# AuthServiceProvider.php

```php
<?php

use Illuminate\Support\Facades\DB;

public function boot()
    {
        // Here you may define how you wish users to be authenticated for your Lumen
        // application. The callback which receives the incoming request instance
        // should return either a User instance or null. You're free to obtain
        // the User instance via an API token or any other method necessary.

        $this->app['auth']->viaRequest('api', function ($request) {
            if ($request->header('rivai')) {
                $token = str_replace('Bearer', '', $request->header('rivai'));
                return DB::connection('mysql')->table('users')->where('password', $token)->first();
            }
        });
    }

?>
```

# Tahun_Bulan_Tanggal_Id_create_users_table.php

```php
<?php

public function up(): void
    {
        Schema::create('users', function (Blueprint $table) {
            $table->id();
            $table->string('username');
            $table->string('password');
            $table->timestamps();
        });
    }

?>
```

# DatabaseSeeder.php

```php
<?php

public function run()
    {
        $this->call([
            UserSeeder::class,
        ]);
    }

?>
```

# UserSeeder.php

```php
<?php

use Illuminate\Support\Facades\DB;

class UserSeeder extends Seeder
{
    public function run(): void
    {
        $timestamp = \Carbon\Carbon::now()->toDateTimeString();

        DB::table('users')->insert([
            'username' => 'rivai',
            'password' => 'ganteng banget',
            'created_at' => $timestamp,
            'updated_at' => $timestamp
        ]);
    }
}

?>
```

# web.php

```php
<?php

$router->group(['prefix' => 'api/v1/user', 'middleware' => 'auth'], function () use ($router) {
    $router->get('/', ['uses' => 'UserController@index']);
});

?>
```